In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import scipy.io as sci

def data_prepare_2D(X,Y,N,traj_length):                # regularize trajectories for training
    import numpy as np 
    thr=1e-10
    r = np.array(X).reshape(N,2,i)              
    r = np.diff(r,axis=2)                              # get the increments
    x = r[:,0,:]                                       # get x data
    y = r[:,1,:]                                       # get y data
    sx = np.std(x,axis=1)
    x = (x-np.mean(x,axis=1).reshape(len(x),1)) / np.where(sx>thr,sx,1).reshape(len(x),1)   # normalize x data
    sy = np.std(y,axis=1)
    y = (y-np.mean(y,axis=1).reshape(len(y),1)) / np.where(sy>thr,sy,1).reshape(len(y),1)   # normalize y data
    r = np.concatenate((x,y),axis=1)                   # merge back x and y
    label = Y
    return(r, label)

In [ ]:
###### Building the recurrent neural network #####

model_inference = Sequential()

block_size = 8                                             # Size of the blocks of data points

model_inference.add(LSTM(250,                              # first layer: LSTM of dimension 250
                         return_sequences=True,            # return sequences for the second LSTM layer            
                         recurrent_dropout=0.2,            # recurrent dropout for preventing overtraining
                         input_shape=(None, block_size)))  # input shape
                                                           
model_inference.add(LSTM(50,                               # second layer: LSTM of dimension 50
                        dropout=0,
                        recurrent_dropout=0.2))

model_inference.add(Dense(1))                              # output 

model_inference.compile(optimizer='adam',
                        loss='mse', 
                        metrics=['mae'])

model_inference.summary()                                  # Printing a summary of the built network

In [ ]:
###### Training the recurrent neural network #####

dimension = 2                # 1, 2 or 3 Dimensions
N = 100000                   # Number of trajectories per datasets
traj_length = 225            # Length of the trajectories

batch_sizes = [32, 128, 512, 2048]
dataset_used = [1, 4, 5, 20]
number_epochs = [5, 4, 3, 2]
n = 0

for batch in range(len(batch_sizes)):    
    for repeat in range(dataset_used[batch]):
        data = sci.loadmat(r'data\inference\ ' + str(dimension) + 'D_' + str(traj_length) + '_' + str(n) + '.mat')
        n += 1
        X = data['X'][0][0]
        Y = data['Y'][0][0].reshape(N,)
        x, label = data_prepare(X,Y,N,traj_length)
        x = np.transpose(x.reshape(N,2,traj_length-1),axes = [0,2,1])
        model_inference.fit(x.reshape(N,int(dimension*traj_length/block_size),block_size),
                            label,
                            epochs=number_epochs[batch], 
                            batch_size=batch_sizes[batch],
                            validation_split=0.1,
                            shuffle=True)